<a href="https://colab.research.google.com/github/MatsuTaku/afkjourney-techcard-optimizer/blob/develop/%5BAFKJ%5D%E9%A0%98%E5%9C%B0_%E6%8A%80%E8%A1%93%E3%82%AB%E3%83%BC%E3%83%89%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%BF%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#技術カードシミュレータv1.2 (~Lv8)

カードの組み合わせによる生産アイテムの星数の出現確率を，動的計画法を用いて厳密に計算します

計算済みの結果は[こちら(Notion)](https://www.notion.so/AFK-Journey-24ce0075498480048a45fbc3b1ecacf4?source=copy_link)にある程度まとめてあります

下のフォームを確認し，上部バーの「すべてのセルを実行」を押してください．ページ最下部に結果が表示されます

*推奨：「ドライブにコピー」して利用することで，自身のGoogleDriveに各種変更が保存されるようになります．ただし，最新版を利用する際は[元のページ](https://colab.research.google.com/github/MatsuTaku/afkjourney-techcard-optimizer/blob/main/%5BAFKJ%5D%E9%A0%98%E5%9C%B0_%E6%8A%80%E8%A1%93%E3%82%AB%E3%83%BC%E3%83%89%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%BF%E3%83%BC.ipynb)から再度「ドライブにコピー」してください*

[GitHub](https://github.com/MatsuTaku/afkjourney-techcard-optimizer)

In [ ]:
#@title Definitions
# !pip install numpy --quiet
!pip install tqdm --quiet

from collections import defaultdict
import numpy as np

items_kitchen = (
    'CircleBread',
    'PanCake',
    'Toast',
    'SpicyBread',
    'WineVinegar',
    'Dolma',
    'SaltyDryFruits',
    'SweetAndSpicyDryFruits',
    'ShroomStri-Fry',
    'DriedMushroom',
    'PinkrootStew',
    'UnknownItem(Kitchen)',
)
items_kitchen_jp = (
    '丸パン',
    'パンケーキ',
    'トースト',
    '旨辛パン',
    'ブドウ酢',
    'ブドウの塩漬け',
    '塩漬けドライフルーツ',
    '甘辛ドライフルーツ',
    'スパイシーキノコ炒め',
    'ピンクの干しキノコ',
    'PinkrootStew',
    '未知アイテム(厨房)',
)
items_workshop = (
    'IronCarvingKnife',
    'IromMask',
    'ThinCarvingKnife',
    'WoodenArrow',
    'WoodenBox',
    'CopperPot',
    'ChillguardArmor',
    'StoneArmor',
    'WarmStoneArmor',
    'FlameguardPlate',
    'UnknownItem(Forge)',
)
items_workshop_jp = (
    '鋼の彫刻刀',
    '鋼のマスク',
    '超薄刃彫刻刀',
    '木の矢',
    '木彫りの箱',
    '保温の銅鍋',
    '保温の木の鎧',
    '岩の鎧',
    '温もりの岩の鎧',
    'FlameguardPlate',
    '未知アイテム(鍛造工房)',
)
items_alchemy = (
    'GrowthPotion',
    'Bouquet',
    'Warmdust',
    'CalmingWarmdust',
    'WarmingIncense',
    'CalmwindIncense',
    'UnknownItem(Alchemy)',
)
items_alchemy_jp = (
    '成長剤',
    '安らぎの花束',
    '温もり粉',
    '安らぎの温もり粉',
    '温もりのアロマ石',
    '安らぎのアロマ石',
    '未知アイテム(錬金工房)',
)
levels_for_luminight = {
    'kitchen': {# ['火加減把握Lv.1', '切り分けLv.4', '調味Lv.1', '弱火煮込みLv.1', '発酵']
        1: [0,1,1,0],
        2: [0,1,1,0],
        3: [0,2,1,0],
        4: [1,3,1,0],
        5: [1,4,1,1],
        6: [2,4,1,2],
        7: [3,5,1,2],
        8: [4,5,1,2,1],
        9: [4,5,1,3,1],
        10: [4,6,1,3,1],
        11: [4,6,1,3,1],
    },
    'workshop': {#['熟練鍛造', '鍛造', '点火', '加熱', 'チャージ']
        1: [0,1,1,0,0],
        2: [0,1,1,0,0],
        3: [0,2,1,0,0],
        4: [0,3,1,1,0],
        5: [1,3,1,1,0],
        6: [2,4,1,1,1],
        7: [3,4,1,2,1],
        8: [3,4,1,2,2],
        9: [4,4,1,2,2],
        10: [4,4,1,2,2],
        11: [4,5,1,2,2],
    },
    'alchemy': {#['材料投入', '研磨', '蒸留', '魔力注入', '融合']
        1: [1,1,0,0],
        2: [1,1,0,0],
        3: [1,1,1,0],
        4: [1,2,1,1],
        5: [2,2,1,1],
        6: [2,3,1,1],
        7: [3,3,1,1],
        8: [3,4,1,2,1],
        9: [4,4,1,2,1],
        10: [4,4,1,2,1],
        11: [4,4,1,2,2],
    }
}
card_limits_map = {
    'Kitchen': {# ['火加減把握Lv.1', '切り分けLv.4', '調味Lv.1', '弱火煮込みLv.1', '発酵']
        1: [0,2,1,0],
        2: [0,2,1,0],
        3: [3,3,1,0],
        4: [3,3,1,1],
        5: [4,3,1,2],
        6: [4,3,1,2],
        7: [4,3,1,2,2],
        8: [4,3,1,2,2],
        9: [4,3,1,2,2],#temporary
        10: [4,3,1,2,2],
        },
    'Workshop': {#['熟練鍛造', '鍛造', '点火', '加熱', 'チャージ', '多重鍛造']
        1: [0,2,1,0,0],
        2: [0,2,1,0,0],
        3: [0,2,1,1,0],
        4: [2,3,1,1,0],
        5: [3,3,1,1,1],
        6: [4,3,1,2,2],
        7: [4,3,1,2,2],
        8: [4,3,1,2,2,2],#temporary
        9: [4,3,1,2,2,2],
        10: [4,3,1,2,2,2],
        },
    'Alchemy': {#['材料投入', '研磨', '蒸留', '魔力注入', '融合']
        1: [2,2,0,0],
        2: [2,2,1,0],
        3: [2,2,1,0],
        4: [2,2,1,2],
        5: [2,2,1,2],
        6: [3,3,1,2],
        7: [4,4,1,2],
        8: [4,4,1,2,1],
        9: [4,4,1,2,1],#temporary
        10: [4,4,1,2,1],
        },
}
build_item_all = items_kitchen + items_workshop + items_alchemy
build_item_jp_all = items_kitchen_jp + items_workshop_jp + items_alchemy_jp
print(list(build_item_jp_all))
build_item_jp = 'スパイシーキノコ炒め' #@param ['丸パン', 'パンケーキ', 'トースト', '旨辛パン', 'ブドウ酢', 'ブドウの塩漬け', '塩漬けドライフルーツ', '甘辛ドライフルーツ', 'スパイシーキノコ炒め', 'ピンクの干しキノコ', 'PinkrootStew', '未知アイテム(厨房)', '鋼の彫刻刀', '鋼のマスク', '超薄刃彫刻刀', '木の矢', '木彫りの箱', '保温の銅鍋', '保温の木の鎧', '岩の鎧', '温もりの岩の鎧', 'FlameguardPlate', '未知アイテム(鍛造工房)', '成長剤', '安らぎの花束', '温もり粉', '安らぎの温もり粉', '温もりのアロマ石', '安らぎのアロマ石', '未知アイテム(錬金工房)']

build_item = build_item_all[build_item_jp_all.index(build_item_jp)]
#@markdown ★ランク条件(未知アイテム用)
star_rank_threshold_for_unsupported_item = 1000 #@param {type:"number"}
#@markdown カード枚数(未知アイテム用)
cound_cards_for_unsupported_item = 7 #@param {type:"number"}
# #@markdown 最低制度保証の星数(default=-1)
stars_for_acculate_eval_at_least = -1
COUNT_CARDS = {
    # Kitchen
    'CircleBread': 3,
    'PanCake': 4,
    "Toast": 5,
    'SpicyBread': 5,
    'WineVinegar': 6,
    'Dolma': 4,
    'SweetAndSpicyDryFruits': 7,
    'SaltyDryFruits': 5,
    'ShroomStri-Fry': 6,
    'DriedMushroom': 5,
    'PinkrootStew': [],
    'UnknownItem(Kitchen)': cound_cards_for_unsupported_item,
    # ForgingWorkshop
    'IronCarvingKnife': 3,
    'IromMask': 4,
    'ThinCarvingKnife': 5,
    'WoodenArrow': 5,
    'WoodenBox': 4,
    'CopperPot': 5,
    'ChillguardArmor': 6,
    'StoneArmor': 7,
    'WarmStoneArmor': 7,
    'FlameguardPlate': [],
    'UnknownItem(Forge)': cound_cards_for_unsupported_item,
    # AlchemyWorkshop
    'GrowthPotion': 3,
    'Bouquet': 6,
    'Warmdust': 7,
    'CalmingWarmdust': 7,
    'WarmingIncense': 6,
    'CalmwindIncense': 6,
    'UnknownItem(Alchemy)': cound_cards_for_unsupported_item,
}
STAR_NEEDS = {
    # Kitchen
    'CircleBread': [0,6,8],
    'PanCake': [0,15,25],
    "Toast": [0,102,159],
    'SpicyBread': [0,372,448,635],
    'WineVinegar': [0,1070,1270,1420,1740],
    'Dolma': [0,344,440,462,647],
    'SweetAndSpicyDryFruits': [0,2670,2950,3340,4020],
    'SaltyDryFruits': [0,1040,1230,1350,1640],
    'ShroomStri-Fry': [0,2160,2450,2810,3520],
    'DriedMushroom': [0,2730,3150,4460,5260],
    'PinkrootStew': [],
    'UnknownItem(Kitchen)': [0,star_rank_threshold_for_unsupported_item],
    # ForgingWorkshop
    'IronCarvingKnife': [0,6,8],
    'IromMask': [0,19,23],
    'ThinCarvingKnife': [0, 46, 86],
    'WoodenArrow': [0, 108, 130, 163],
    'WoodenBox': [0, 152, 180, 182, 190],
    'CopperPot': [0, 566, 675, 797, 1040],
    'ChillguardArmor': [0,762,873,974,1190],
    'StoneArmor': [0,2210,2600,2920,3770],
    'WarmStoneArmor': [0,13700,18100,29600,56900],
    'FlameguardPlate': [],
    'UnknownItem(Forge)': [0,star_rank_threshold_for_unsupported_item],
    # AlchemyWorkshop
    'GrowthPotion': [0,12,27],
    'Bouquet': [0,518,575,637,744],
    'Warmdust': [0,1510,1630,1760,2040],
    'CalmingWarmdust': [0,2150,2350,2520,2860],
    'WarmingIncense': [0,1290,1430,1560,1870],
    'CalmwindIncense': [0,2070,2340,2670,3420],
    'UnknownItem(Alchemy)': [0,star_rank_threshold_for_unsupported_item],
}

#@markdown 厨房
kitchen_level = 8 #@param {type:"number", label:"厨房"}
#@markdown - 火加減把握
heat_controll_card_level = 4 #@param {type:"number", label:"火加減把握"}
#@markdown - 切り分け
cut_card_level = 5 #@param {type:"number", label:"切り分け"}
cook_card_level = 1
#@markdown - 弱火煮込み
low_heat_card_level = 3 #@param {type:"number", label:"弱火煮込み"}
#@markdown - 発酵
ferment_card_level = 2 #@param {type:"number", label:"発酵"}

#@markdown 鍛造工房
forge_level = 8 #@param {type:"number", label:"鍛造工房"}
#@markdown - 熟練鍛造
skilled_forging_level = 4 #@param {type:"number", label:"熟練鍛造"}
#@markdown - 鍛造
forging_level = 4 #@param {type:"number", label:"鍛造"}
ignition_level = 1
#@markdown - 加熱
burning_level = 2 #@param {type:"number", label:"加熱"}
#@markdown - チャージ
charge_level = 2 #@param {type:"number", label:"チャージ"}
#@markdown - 多重鍛造
multi_forge_level = 1 #@param {type:"number", label:"多重鍛造"}

#@markdown 錬金工房
alchemy_level = 8 #@param {type:"number", label:"錬金工房"}
#@markdown - 材料投入
ingredients_level = 4 #@param {type:"number", label:"材料投入"}
#@markdown - 研磨
grind_level = 4 #@param {type:"number", label:"研磨"}
distillation_level = 1 # 蒸留
#@markdown - 魔力注入
enchant_level = 2 #@param {type:"number", label:"魔力注入"}
#@markdown - 融合
fuse_level = 1 #@param {type:"number", label:"融合"}

In [ ]:
# @title Algorithms
# DP_TABLE_SIZE = 300
DP_RANK_LIM = STAR_NEEDS[build_item][stars_for_acculate_eval_at_least]
import math
DP_RANK_SQRT = math.ceil(math.sqrt(DP_RANK_LIM))
TABLE_MODE_SQUARE = 'TABLE_MODE_SQUARE'
TABLE_MODE_FIT = 'TABLE_MODE_FIT'
TABLE_MODE = TABLE_MODE_SQUARE
DP_SQRTSIZE = 3
if TABLE_MODE == TABLE_MODE_SQUARE:
    DP_RANK_SQRT *= DP_SQRTSIZE
elif TABLE_MODE == TABLE_MODE_FIT:
    DP_RANK_LIM = DP_RANK_SQRT * DP_SQRTSIZE
# DP_RANK_LIM = max(DP_RANK_SQRT*3, 100)
DP_RANK_BUFFER = 10
DP_RANK_LIM += DP_RANK_BUFFER
DP_RANK_SQRT += DP_RANK_BUFFER
DP_SIZE = DP_RANK_LIM if TABLE_MODE == TABLE_MODE_FIT else DP_RANK_SQRT if TABLE_MODE == TABLE_MODE_SQUARE else 0
# DP_SHAPE = (DP_TABLE_SIZE, DP_TABLE_SIZE)
ADD_TYPE_RANDOM = "ADD_TYPE_RANDOM"
ADD_TYPE_ALL = "ADD_TYPE_ALL"
ADD_TYPE_MAX = "ADD_TYPE_MAX"
ADD_TYPE_MIN = "ADD_TYPE_MIN"

def get_dp_size_block(x):
    return DP_RANK_LIM//x+1-1 # [1:DP_RANK_LIM//x+1]
import copy
class DPT:
    SHAPE = [(DP_RANK_SQRT, DP_RANK_LIM), (DP_RANK_LIM-DP_RANK_SQRT, DP_RANK_SQRT)] if TABLE_MODE == TABLE_MODE_FIT else [(DP_RANK_SQRT, DP_RANK_SQRT)] if TABLE_MODE == TABLE_MODE_SQUARE else []
    def __init__(self, initial_dp=None):
        dps = []
        if initial_dp is None:
            dps = [np.zeros(s) for s in DPT.SHAPE]
        else:
            dps = copy.deepcopy(initial_dp)
        self.dp = dps
    def sum(self):
        return sum(np.sum(v) for v in self.dp)
    def __len__(self):
        return len(self.dp)
    def __getitem__(self, key):
        return self.dp[key]
    def __setitem__(self, key, value):
        self.dp[key] = value
    def __copy__(self):
        return DPT(self.dp)
    def copy(self):
        return DPT(self.dp)
    def __iadd__(self, rhs):
        if isinstance(rhs, DPT):
            for i in range(len(self.dp)):
                self[i] += rhs[i]
            return self
        else:
            for i in range(len(self.dp)):
                self[i] += rhs
            return self
    def __add__(self, rhs):
        t = self.copy()
        return t.__iadd__(rhs)
    def __imul__(self, rhs):
        if isinstance(rhs, DPT):
            for i in range(len(self.dp)):
                self[i] *= rhs[i]
            return self
        else:
            for i in range(len(self.dp)):
                self[i] *= rhs
            return self
    def __mul__(self, rhs):
        t = self.copy()
        return t.__imul__(rhs)
    def __itruediv__(self, rhs):
        if isinstance(rhs, DPT):
            for i in range(len(self.dp)):
                self[i] /= rhs[i]
            return self
        else:
            for i in range(len(self.dp)):
                self[i] /= rhs
            return self
    def __truediv__(self, rhs):
        t = self.copy()
        return t.__itruediv__(rhs)
def get_zero_dp_blocks():
    return DPT()
def zeropad(dp, shape):
    return np.pad(dp, shape, mode='constant', constant_values=0)
def dsadd(a,b):
    sa = a.shape
    sb = b.shape
    s = tuple(max(u,v) for u,v in zip(sa,sb))
    if a.shape!=s:
        a = zeropad(a, ((0,s[0]-sa[0]),(0,s[1]-sa[1])))
    if b.shape!=s:
        b = zeropad(b, ((0,s[0]-sb[0]),(0,s[1]-sb[1])))
    return a+b

def dp_shift_b(dp, shift_shape):
    if shift_shape[0]==0 and shift_shape[1]==0:
        return dp.copy()
    ndp = dp.copy()
    psum = dp.sum()
    def check_sum():
        sum = ndp.sum()
        if not abs(psum-sum)<1e-6:
            print("dp_shift_b", shift_shape)
            print(psum, sum)
        assert(abs(psum-sum)<1e-6)

    if shift_shape[1]>0:
        for i in range(len(ndp)):
            ndp[i] = zeropad(ndp[i], ((0,0),(shift_shape[1],0)))
        check_sum()
    elif shift_shape[1]<0:
        dy = -shift_shape[1]
        for i in range(len(ndp)):
            t = zeropad(ndp[i], ((0,0),(0, dy)))
            t[:,dy] += np.sum(t[:,:dy], axis=1)
            ndp[i] = t[:,dy:]
        check_sum()

    if shift_shape[0]>0:
        dx = shift_shape[0]
        for i in range(len(ndp)):
            ndp[i] = zeropad(ndp[i], ((dx,0),(0,0)))
        if TABLE_MODE==TABLE_MODE_FIT:
            ndp[1][:dx] += ndp[0][-dx:,:ndp[1].shape[1]]
            ndp[0][-dx:,:ndp[1].shape[1]] = 0
            check_sum()
            ndp[1][-dx-1] += np.sum(ndp[1][-dx:], axis=0)
            ndp[1] = ndp[1][:-dx]
            check_sum()
        ndp[0][-dx-1] += np.sum(ndp[0][-dx:], axis=0)
        ndp[0] = ndp[0][:-dx]
    elif shift_shape[0]<0:
        dx = -shift_shape[0]
        ndp[0] = zeropad(ndp[0], ((0,dx),(0,0)))
        ndp[0][dx] += np.sum(ndp[0][:dx], axis=0)
        ndp[0] = ndp[0][dx:]
        if TABLE_MODE==TABLE_MODE_FIT:
            ndp[1] = zeropad(ndp[1], ((0,dx),(0,0)))
            ndp[0][-dx:,:ndp[1].shape[1]] += ndp[1][:dx]
            ndp[1] = ndp[1][dx:]
            check_sum()

    # print(ndp[0].shape, ndp[1].shape)
    if TABLE_MODE==TABLE_MODE_FIT:
        ndp[0][:,DP_RANK_LIM-1] += np.sum(ndp[0][:,DP_RANK_LIM:], axis=1)
        ndp[0] = ndp[0][:,:DP_RANK_LIM]
        ndp[1][:,DP_RANK_SQRT-1] += np.sum(ndp[1][:,DP_RANK_SQRT:], axis=1)
        ndp[1] = ndp[1][:,:DP_RANK_SQRT]
    elif TABLE_MODE==TABLE_MODE_SQUARE:
        ndp[0][:,DP_RANK_SQRT-1] += np.sum(ndp[0][:,DP_RANK_SQRT:], axis=1)
        ndp[0] = ndp[0][:,:DP_RANK_SQRT]

    check_sum()
    # print(ndp[0].shape, ndp[1].shape)
    for i,s in enumerate(DPT.SHAPE):
        if ndp[i].shape!=s:
            print("dp_shift_b", shift_shape, ndp[i].shape, s)
        assert(ndp[i].shape == s)
    return ndp

def random_shift(dp, add_unit):
    xadd = dp_shift_b(dp, (add_unit,0))
    yadd = dp_shift_b(dp, (0, add_unit))
    return (xadd+yadd)/2

def all_shift(dp, add_unit):
    return dp_shift_b(dp, (add_unit,add_unit))

iej_mask = get_zero_dp_blocks()
igj_mask = get_zero_dp_blocks()
ilj_mask = get_zero_dp_blocks()
for i in range(DP_RANK_SQRT):
    x = i+1
    # l = get_dp_size_block(x)
    # if i<l:
    iej_mask[0][i][i] = 1
    igj_mask[0][i][:i] = 1
    ilj_mask[0][i][i+1:] = 1
if TABLE_MODE == TABLE_MODE_FIT:
    igj_mask[1] = 1
def mask_dp(dp, mask):
    return dp * mask

def max_shift(dp, add_unit):
    ilj = dp_shift_b(mask_dp(dp,ilj_mask), (0,add_unit))
    igj = dp_shift_b(mask_dp(dp,igj_mask), (add_unit,0))
    iej = random_shift(mask_dp(dp,iej_mask), add_unit)
    return ilj+iej+igj

def min_shift(dp, add_unit):
    ilj = dp_shift_b(mask_dp(dp,ilj_mask), (add_unit,0))
    igj = dp_shift_b(mask_dp(dp,igj_mask), (0,add_unit))
    iej = random_shift(mask_dp(dp,iej_mask), add_unit)
    return ilj+iej+igj

def max_min_shift(dp, max_shift, min_shift):
    ilj = dp_shift_b(mask_dp(dp,ilj_mask), (min_shift,max_shift))
    igj = dp_shift_b(mask_dp(dp,igj_mask), (max_shift,min_shift))
    iejm = mask_dp(dp,iej_mask)
    iejx = dp_shift_b(iejm, (min_shift,max_shift))
    iejy = dp_shift_b(iejm, (max_shift,min_shift))
    return ilj+(iejx+iejy)/2+igj

def double_x(dp):
    ret = get_zero_dp_blocks()
    for i in range(DP_RANK_SQRT):
        x = i+1
        ni = x*2-1
        if TABLE_MODE==TABLE_MODE_FIT:
            if ni<DP_RANK_SQRT:
                ret[0][ni] += dp[0][i]
            else:
                nni = min(ni,DP_RANK_LIM-1)-DP_RANK_SQRT
                ret[1][nni] += dp[0][i,:DP_RANK_SQRT]
                ret[1][nni,-1] += np.sum(dp[0][i,DP_RANK_SQRT:])
        elif TABLE_MODE==TABLE_MODE_SQUARE:
            nni = min(ni, DP_RANK_SQRT-1)
            ret[0][nni] += dp[0][i]
    if TABLE_MODE==TABLE_MODE_FIT:
        for i in range(DP_RANK_SQRT, DP_RANK_LIM):
            ni = x*2-1
            nni = min(ni,DP_RANK_LIM)-DP_RANK_SQRT
            ret[1][nni] += dp[1][i-DP_RANK_SQRT,:DP_RANK_SQRT]
            ret[1][nni,-1] += np.sum(dp[1][i-DP_RANK_SQRT,DP_RANK_SQRT:])
    psum = dp.sum()
    sum = ret.sum()
    d = abs(psum-sum)
    if not d<1e-6:
        print("double_x", psum, sum)
    assert(d<1e-6)
    return ret
def double_y(dp):
    ret = get_zero_dp_blocks()
    if TABLE_MODE==TABLE_MODE_FIT:
        for j in range(DP_RANK_LIM):
            y = j+1
            nj = min(y*2, DP_RANK_LIM)-1
            ret[0][:,nj] += dp[0][:,j]
            if j<DP_RANK_SQRT:
                nnj = min(nj, DP_RANK_SQRT-1)
                ret[1][:,nnj] += dp[1][:,j]
    elif TABLE_MODE==TABLE_MODE_SQUARE:
        for j in range(DP_RANK_SQRT):
            y = j+1
            nj = min(y*2, DP_RANK_SQRT)-1
            ret[0][:,nj] += dp[0][:,j]
    psum = dp.sum()
    sum = ret.sum()
    d = abs(psum-sum)
    if not d<1e-6:
        print("double_y", psum, sum)
    assert(d<1e-6)
    return ret

def min_double(dp):
    ret = get_zero_dp_blocks()
    ilj = dp*ilj_mask
    igj = dp*igj_mask
    iej = dp*iej_mask
    ilj += iej/2
    igj += iej/2
    return double_y(igj) + double_x(ilj)

def max_double(dp):
    ret = get_zero_dp_blocks()
    ilj = dp*ilj_mask
    igj = dp*igj_mask
    iej = dp*iej_mask
    ilj += iej/2
    igj += iej/2
    return double_y(ilj) + double_x(igj)

def test_sum_equal(dp):
    fs = [lambda x: dp_shift_b(x, (8,8)),
          lambda x: dp_shift_b(x, (-8,-8)),
          lambda x: dp_shift_b(x, (-8,8)),
          lambda x: dp_shift_b(x, (8,-8)),
          double_x, double_y]
    for f in fs:
        ret = f(dp)
        assert(abs(ret.sum()-dp.sum())<1e-6)
test_sum_equal(get_zero_dp_blocks()+1)

In [ ]:
# @title Kitchen Card Classes
class Card:
    def __init__(self, level, name, jname, card_type=None):
        self.level = level
        self.name = name
        self.jname = jname
        self.card_type = card_type

    def update_dp(self, dp_old, stack_table, idx, n):
        return dp_old.copy()

HEAT_CONTROLL_ID = 0
CUT_ID = 1
COOK_ID = 2
LOW_HEAT_ID = 3
FERMENT_ID = 4

HEAT_CONTROLL_STACK_LIMIT = 7

class CutCard(Card):
    ADD_UNIT_MIN = [0,1,2,3,4,8,10]
    ADD_UNIT_MAX = [0,1,2,3,8,12,15]
    def __init__(self, level):
        super().__init__(level, 'CutCard', '切り分け')

    def update_dp(self, dp_old, stack_table, idx, n):
        pdp = dp_old.copy()
        lo = CutCard.ADD_UNIT_MIN[self.level]
        hi = CutCard.ADD_UNIT_MAX[self.level]
        if build_item == 'Dolma':
            lo = hi
        # ret = np.zeros_like(dp_old)
        ret = get_zero_dp_blocks()
        for d in range(lo, hi+1):
            ret += random_shift(pdp, d)
            # dx = dp_add_shift(pdp, (d,0))
            # dy = dp_add_shift(pdp, (0,d))
            # ret += dx
            # ret += dy
        return ret / (hi+1-lo)

class CookCard(Card):
    def __init__(self, level):
        super().__init__(level, 'CookCard', '調味')

    def update_dp(self, dp_old, stack_table, idx, n):
        pdp = dp_old.copy()
        # mx = [2,1]
        # my = [1,2]
        # ret = np.zeros_like(dp_old)
        ret = get_zero_dp_blocks()
        ret += double_x(pdp)
        ret += double_y(pdp)
        # for i in range(DP_TABLE_SIZE):
        #     for j in range(DP_TABLE_SIZE):
        #         for k in range(2):
        #             ni = i*mx[k]
        #             nj = j*my[k]
        #             ni = min(ni,DP_TABLE_SIZE-1)
        #             nj = min(nj,DP_TABLE_SIZE-1)
        #             ret[ni,nj] += pdp[i,j]
        return ret/2

CMB_LIM = 10
fact = [1 for _ in range(CMB_LIM)]
for i in range(1,CMB_LIM):
    fact[i] = fact[i-1]*i

def cmb(n,r):
    if n<r:
        return 0
    return fact[n]/(fact[r]*fact[n-r])

class LowHeat(Card):
    ADD_UNIT = [0,4,2,4]
    ADD_TYPE = ["-", ADD_TYPE_RANDOM, ADD_TYPE_ALL, ADD_TYPE_ALL]
    def __init__(self, level):
        self.add_unit = LowHeat.ADD_UNIT[level]
        self.add_type = LowHeat.ADD_TYPE[level]
        super().__init__(level, 'LowHeat', '弱火煮込み')

    def hooked_dp(self, dp_old):
        pdp = dp_old.copy()
        if self.add_type == ADD_TYPE_RANDOM:
            return random_shift(pdp, self.add_unit)
        elif self.add_type == ADD_TYPE_ALL:
            return all_shift(pdp, self.add_unit)

class HeatControllCard(Card):
    ADD_UNIT = [0,3,4,6,8,10]
    def __init__(self, level):
        self.add_unit = HeatControllCard.ADD_UNIT[level]
        super().__init__(level, 'HeatControllCard', '火加減把握')
        self.continue_prop = 0.5

    def update_dp(self, dp_old, stack_table, idx, n):
        dp = [get_zero_dp_blocks() for _ in range(20)]
        dp[0] = dp_old.copy()
        sts = [stack_table.copy() for i in range(20)]
        low_heat = LowHeat(low_heat_card_level)
        for k in range(1,20):
            st = sts[k]
            p_cont = self.continue_prop if k>1+(Ferment(ferment_card_level).force_count if st[FERMENT_ID]>0 else 0) else 1
            pdp = dp[k-1] * p_cont
            dp[k] = random_shift(pdp, self.add_unit)
            for low_heats in range(st[LOW_HEAT_ID]):
                dp[k] = low_heat.hooked_dp(dp[k])
            if k>=1:
                dp[k-1] *= (1-p_cont)
            if k+1<20:
                sts[k+1][HEAT_CONTROLL_ID] = sts[k][HEAT_CONTROLL_ID]+1
        if build_item == 'DriedMushroom':
            ret = [[sts[k], dp[k]] for k in range(1,20)]
            hci = 1
            while hci<len(ret) and ret[hci][0][HEAT_CONTROLL_ID]<=HEAT_CONTROLL_STACK_LIMIT:
                hci += 1
            for i in range(hci, len(ret)):
                ret[hci-1][1] += ret[i][1]
            return ret[:hci]
        else:
            ret = get_zero_dp_blocks()
            for k in range(1,20):
                ret += dp[k]
            return ret

class Ferment(Card):
    FORCE_COUNT = [0,1,2]
    def __init__(self, level):
        super().__init__(level, 'Ferment', '発酵')
        self.force_count = Ferment.FORCE_COUNT[level]

In [ ]:
# @title Forging Workshop Card Classes

SKILLED_FORGING_ID = 0
FORGING_ID = 1
IGNITION_ID = 2
BURNING_ID = 3
CHARGE_ID = 4
MULTIFORGE_ID = 5

COPPER_POT_BONUS = 0.3
class SkilledForging(Card):
    ADD_UNIT = [0,3,4,4,5]
    BONUS_UNIT = [0,3,3,4,4]
    def __init__(self, level):
        self.add_unit = SkilledForging.ADD_UNIT[level]
        self.bonus_unit = SkilledForging.BONUS_UNIT[level]
        super().__init__(level, 'SkilledForging', '熟練鍛造', 'Artisan')
    def update_dp(self, dp_old, stack_table, idx, n):
        ret = dp_old.copy()
        def act_unit(ret):
            add_unit = self.add_unit
            add_unit += self.bonus_unit * stack_table[SKILLED_FORGING_ID]
            add_unit += Burning(burning_level).bonus_unit * stack_table[BURNING_ID]
            f = all_shift if stack_table[CHARGE_ID]>0 else random_shift
            return f(ret, add_unit)
        act_count = 1
        if stack_table[MULTIFORGE_ID]>0:
            act_count += MultiForge(multi_forge_level).multi_count
            stack_table[MULTIFORGE_ID]=0
        for i in range(act_count):
            ret = act_unit(ret)
            if i<act_count-1:
                stack_table[SKILLED_FORGING_ID] += 1
        if build_item == 'CopperPot':
            ret = ret * (1-COPPER_POT_BONUS) + act_unit(ret) * COPPER_POT_BONUS
        if stack_table[CHARGE_ID]>0:
            stack_table[CHARGE_ID] -= 1
        return ret
class Forging(Card):
    ADD_UNIT = [0,1,2,3,5,10]
    def __init__(self, level):
        self.add_unit = Forging.ADD_UNIT[level]
        super().__init__(level, 'Forging', '鍛造', 'Artisan')
    def update_dp(self, dp_old, stack_table, idx, n):
        ret = dp_old.copy()
        woodenbox_bonus = (build_item=='WoodenBox' and stack_table[FORGING_ID]==0)#Ambiguous condition
        def act_unit(ret):
            add_unit = self.add_unit
            add_unit += Burning(burning_level).bonus_unit * stack_table[BURNING_ID]
            f = random_shift
            if stack_table[CHARGE_ID]>0 or woodenbox_bonus:
                f = all_shift
            return f(ret, add_unit)
        act_count = 1
        if stack_table[MULTIFORGE_ID]>0:
            act_count += MultiForge(multi_forge_level).multi_count
            stack_table[MULTIFORGE_ID]=0
        for i in range(act_count):
            ret = act_unit(ret)
            if i<act_count-1:
                stack_table[FORGING_ID] += 1
        if stack_table[CHARGE_ID]>0:
            stack_table[CHARGE_ID] -= 1
        return ret
class Ignition(Card):
    ADD_UNIT = [0,2]
    def __init__(self, level):
        self.add_unit = Ignition.ADD_UNIT[level]
        super().__init__(level, 'Ignition', '点火')
    def update_dp(self, dp_old, stack_table, idx, n):
        pdp = dp_old.copy()
        return random_shift(pdp, self.add_unit)
class Burning(Card):
    BONUS_UNIT = [0,3,6,10]
    def __init__(self, level):
        self.bonus_unit = Burning.BONUS_UNIT[level]
        super().__init__(level, 'Burning', '加熱')
class Charge(Card):
    CHARGE_COUNT = [0,1,3,10000]
    def __init__(self, level):
        super().__init__(level, 'Charge', 'チャージ')
        self.charge_count = Charge.CHARGE_COUNT[level]
class MultiForge(Card):
    MULTI_COUNT = [0,2]
    def __init__(self, level):
        super().__init__(level, 'Multi-Forge', '多重鍛造')
        self.multi_count = MultiForge.MULTI_COUNT[level]
    def update_dp(self, dp_old, stack_table, idx, n):
        ret = dp_old.copy()
        return ret
    def hooked_dp(self, dp_old, stack_table, idx, n, cur_card):
        ret = dp_old.copy()
        assert( stack_table[MULTIFORGE_ID]>0 and cur_card.card_type == 'Artisan')
        for t in range(self.multi_count):
            ret = cur_card.update_dp(ret,stack_table,idx,n)
        stack_table[MULTIFORGE_ID] = 0
        return ret

In [ ]:
# @title Alchemy Workshop Card Classes
INGREDIENTS_ID = 0
GRIND_ID = 1
DISTILL_ID = 2
ENCHANT_ID = 3
FUSE_ID = 4

def alchemy_revealed_before(ret, stack_table):
    ret = Enchant(enchant_level).hooked_dp(ret, stack_table)
    return ret

def alchemy_revealed_after(ret):
    if build_item == 'Warmdust':
        ret = min_shift(ret, 1)
    if build_item == 'CalmingWarmdust':
        ret = max_shift(ret, 3)
    return ret

class Ingredients(Card):
    ADD_UNIT = [0,4,6,10,15]
    MINUS_UNIT = [0,0,0,2,3]
    def __init__(self, level):
        super().__init__(level, 'Ingredients', '材料投入')
        self.add_type = ADD_TYPE_MAX
        self.add_unit = Ingredients.ADD_UNIT[level]
        self.minus_unit = Ingredients.MINUS_UNIT[level]
    def update_dp(self, dp_old, stack_table, idx, n):
        ret = dp_old.copy()
        def act_unit(ret):
            ret = alchemy_revealed_before(ret, stack_table)
            ret = max_min_shift(ret, self.add_unit, -self.minus_unit)
            # ret = max_add_min_minus(ret, self.add_unit, self.minus_unit)
            ret = alchemy_revealed_after(ret)
            return ret
        ret = act_unit(ret)
        if build_item == 'CalmwindIncense':
            stack_table[INGREDIENTS_ID] += 1
            ret = act_unit(ret)
        return ret

class Grind(Card):
    ADD_UNIT = [0,2,3,4,6]
    MINUS_UNIT = [0,0,0,2,3]
    def __init__(self, level):
        super().__init__(level, 'Grind', '研磨')
        self.add_unit = Grind.ADD_UNIT[level]
        self.minus_unit = Grind.MINUS_UNIT[level]
    def update_dp(self, dp_old, stack_table, idx, n):
        ret = dp_old.copy()
        def act_unit(ret):
            ret = alchemy_revealed_before(ret, stack_table)
            ret = max_min_shift(ret, -self.minus_unit, self.add_unit)
            # ret = min_add_max_minus(ret, self.add_unit, self.minus_unit)
            ret = alchemy_revealed_after(ret)
            return ret
        ret = act_unit(ret)
        if build_item == 'WarmingIncense':
            stack_table[GRIND_ID] += 1
            ret = act_unit(ret)
        return ret

class Distill(Card):
    def __init__(self, level):
        super().__init__(level, 'Distill', '蒸留')
    def update_dp(self, dp_old, stack_table, idx, n):
        ret = dp_old.copy()
        ret = alchemy_revealed_before(ret, stack_table)
        ret = max_double(ret)
        ret = alchemy_revealed_after(ret)
        return ret

class Enchant(Card):
    ADD_UNIT = [0,8,16]
    BONUS_MINUS_UNIT = [0,1,1]
    BONUS_MINUS_TYPE = ["-", ADD_TYPE_RANDOM, ADD_TYPE_ALL]
    def __init__(self, level):
        super().__init__(level, 'Enchant', '魔力注入')
        self.add_unit = Enchant.ADD_UNIT[level]
        self.bonus_minus_unit = Enchant.BONUS_MINUS_UNIT[level]
        self.bonus_minus_type = Enchant.BONUS_MINUS_TYPE[level]
    def update_dp(self, dp_old, stack_table, idx, n):
        ret = dp_old.copy()
        ret = alchemy_revealed_before(ret, stack_table)
        ret = min_shift(ret, self.add_unit)
        ret = alchemy_revealed_after(ret)
        return ret
    def hooked_dp(self, dp_old, stack_table):
        ret = dp_old.copy()
        f = random_shift if self.bonus_minus_type == ADD_TYPE_RANDOM else all_shift if self.bonus_minus_type == ADD_TYPE_ALL else None
        for i in range(stack_table[ENCHANT_ID]):
            ret = f(ret, -self.bonus_minus_unit)
        return ret

class Fuse(Card):
    DP_MASK = get_zero_dp_blocks()
    DP_MASK_R = get_zero_dp_blocks()
    def __init__(self, level):
        super().__init__(level, 'Fuse', '融合')
        self.add_unit = 5
        self.diff_th = 10
    def update_dp(self, dp_old, stack_table, idx, n):
        return dp_old.copy()
_fuse = Fuse(fuse_level)
for i in range(DP_SIZE):
    x = i+1
    lo = x - _fuse.diff_th + 1
    hi = x + _fuse.diff_th
    lo = max(lo, 1)
    hi = min(hi, DP_SIZE+1)
    Fuse.DP_MASK[0][i][lo-1:hi-1] = 1
Fuse.DP_MASK_R = Fuse.DP_MASK * -1 + 1
assert((Fuse.DP_MASK + Fuse.DP_MASK_R).sum() == (get_zero_dp_blocks()+1).sum())

In [ ]:
# @title Calculation Prepare
card_classes_kitchen = [
    HeatControllCard(heat_controll_card_level),
    CutCard(cut_card_level),
    CookCard(cook_card_level),
    LowHeat(low_heat_card_level),
    Ferment(ferment_card_level),
]
card_classes_workshop = [
    SkilledForging(skilled_forging_level),
    Forging(forging_level),
    Ignition(ignition_level),
    Burning(burning_level),
    Charge(charge_level),
    MultiForge(multi_forge_level),
]
card_classes_alchemy = [
    Ingredients(ingredients_level),
    Grind(grind_level),
    Distill(distillation_level),
    Enchant(enchant_level),
    Fuse(fuse_level),
]
card_classes_map = {
    'Kitchen': card_classes_kitchen,
    'Workshop': card_classes_workshop,
    'Alchemy': card_classes_alchemy,
}

In [ ]:
# @title Calculation
count_cards = COUNT_CARDS[build_item]
workplace = 'Kitchen' if build_item in items_kitchen else 'Workshop' if build_item in items_workshop else 'Alchemy' if build_item in items_alchemy else None
workplace_jp = {
    'Kitchen': '厨房',
    'Workshop': '鍛造工房',
    'Alchemy': '錬金工房',
}
card_classes = card_classes_map[workplace]
workplace_level = {
    'Kitchen': kitchen_level,
    'Workshop': forge_level,
    'Alchemy': alchemy_level,
}
card_limits = card_limits_map[workplace][workplace_level[workplace]]

def act_production_completed(dp, stack_table):
    if build_item == 'CalmingStoneArmor':
        print("act_production_completed", build_item)
        if stack_table[SKILLED_FORGING_ID] + stack_table[FORGING_ID] >= 6:
            return all_shift(dp, 3)
    if build_item == 'DriedMushroom':
        if stack_table[HEAT_CONTROLL_ID] >= 7:
            return all_shift(dp, 3)
    if workplace == 'Alchemy':
        if stack_table[FUSE_ID] > 0:
            dp = dp*Fuse.DP_MASK_R + all_shift(dp*Fuse.DP_MASK, Fuse(fuse_level).add_unit * stack_table[FUSE_ID])
    return dp

import itertools
def for_perm_list(caps):
    ord = []
    for i,c in enumerate(caps):
        ord += [i for _ in range(c)]
    for perm in itertools.permutations([i for i in range(count_cards)]):
        yield [ord[i] for i in perm]

card_set = [-1 for _ in range(len(card_limits))]
def for_card_set(d):
    if sum(card_set)>count_cards:
        return
    if d==len(card_set):
        if sum(card_set)==count_cards:
            yield card_set.copy()
        return
    for k in range(card_limits[d]+1):
        card_set[d] = k
        yield from for_card_set(d+1)
    card_set[d] = -1

star_needs = STAR_NEEDS[build_item]
star_masks = [get_zero_dp_blocks() for _ in range(len(star_needs))]
dpsz = DP_RANK_LIM if TABLE_MODE==TABLE_MODE_FIT else DP_RANK_SQRT if TABLE_MODE==TABLE_MODE_SQUARE else 0
for i in range(dpsz):
    x = i+1
    infs = [(t-1)//x+1 for t in star_needs] + [dpsz+1]
    infs[0] = 1
    b = 0 if i<DP_RANK_SQRT else 1
    if i==DP_RANK_SQRT:
        infs[-1] = DP_RANK_SQRT+1
    ni = i if b==0 else i-DP_RANK_SQRT
    for c in range(len(star_needs)):
        star_masks[c][b][ni, infs[c]-1:infs[c+1]-1] = 1
def get_probs(tb):
    res = [(tb * star_masks[i]).sum() for i in range(len(star_masks))]
    return res

EPS = 1e-6
def dp_for_remain_cards(deck):
    remain_table = deck.copy()
    dp = defaultdict(lambda: defaultdict(get_zero_dp_blocks))
    idpt = get_zero_dp_blocks()
    idpt[0][1-1,1-1] = 1
    dp[tuple(deck)][tuple(0 for _ in range(len(deck)))] = idpt
    assert(sum(deck) == count_cards)
    for t in range(count_cards):
        pdp = dp
        dp = defaultdict(lambda: defaultdict(get_zero_dp_blocks))
        nsum = 0
        for remain_table,ddp in pdp.items():
            # print("R", remain_table)
            for stack_table, dddp in ddp.items():
                # if dddp.sum() < 1e-6:
                #     continue
                # print("S", stack_table)
                for ci in range(len(deck)):
                    # print(ci)
                    if remain_table[ci]==0:
                        continue
                    assert(sum(remain_table) == count_cards-t)
                    select_prop = remain_table[ci]/sum(remain_table)
                    cstack_table = list(stack_table)
                    cremain_table = list(remain_table)
                    update_ret = card_classes[ci].update_dp(dddp*select_prop, cstack_table, t, count_cards)
                    cremain_table[ci] -= 1
                    if isinstance(update_ret, DPT):
                        ndpt = update_ret
                        cstack_table[ci] += 1
                        if card_classes[ci].name == 'Charge':
                            cstack_table[ci] = card_classes[ci].charge_count
                        # print(cremain_table, cstack_table, np.sum(ndpt))
                        dp[tuple(cremain_table)][tuple(cstack_table)] += ndpt
                        if not abs(dddp.sum()*select_prop-ndpt.sum())<1e-6:
                            print(card_classes[ci].name)
                            print(dddp.sum()*select_prop, ndpt.sum())
                        assert(abs(dddp.sum()*select_prop-ndpt.sum())<1e-6)
                        nsum += ndpt.sum()
                    else:
                        cnsum = 0
                        for st,ndpt in update_ret:
                            cnsum += ndpt.sum()
                            st[ci] += 1
                            if card_classes[ci].name == 'Charge':
                                st[ci] = card_classes[ci].charge_count
                            # print(cremain_table, st, np.sum(ndpt))
                            dp[tuple(cremain_table)][tuple(st)] += ndpt
                        if not abs(dddp.sum()*select_prop-cnsum)<1e-6:
                            print(card_classes[ci].name)
                            print(dddp.sum()*select_prop, cnsum)
                        assert(abs(dddp.sum()*select_prop-cnsum)<1e-6)
                        nsum += cnsum
        # print(t, nsum)
        assert(abs(nsum-1) < 1e-3)
    alldp = get_zero_dp_blocks()
    acc_max_d = 0
    eps = 1e-6
    for ddp in dp.values():
        for stack_table,dddp in ddp.items():
            ret = act_production_completed(dddp, stack_table)
            alldp += ret
    #         for i in range(len(dddp.dp)):
    #             for x in range(dddp[i].shape[0]):
    #                 for y in range(dddp[i].shape[1]):
    #                     if dddp[i][x,y]>eps:
    #                         acc_max_d = max(acc_max_d, y+1)
    # print(acc_max_d, DP_RANK_SQRT, DP_RANK_LIM)
    assert(abs(alldp.sum()-1) < 1e-3)
    return alldp

results = []
from tqdm.notebook import tqdm
for cmb in tqdm(list(for_card_set(0))):
    sdp = dp_for_remain_cards(cmb)
    results.append((tuple(cmb), get_probs(sdp)))

results.sort(key=lambda x: x[1])

print(build_item_jp,
      workplace_jp[workplace], 'Lv.', workplace_level[workplace],
      'カード数', count_cards,
      '★条件', star_needs)
print('カード一覧:', [c.jname+'Lv.'+str(c.level) for c in card_classes])
print('カード内訳 |', '★ランク確率[%]')
for cs, res in results:
    print(list(cs), '|', list(int(v) for v in (np.round(np.array(res)*100)).astype(int)))